In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [3]:
import numpy as np
from Scratch import metadata_loader

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 10
md = md[md["distance"] == max_distance]
md = md.sort_values(by='backend_name', ascending=False)

md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
51,2023-10-29 14:43:41.752908+01:00,bigger_rep_codes,ibm_sherbrooke,cmz633cvayrg008sp2a0,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,_is_hex=True,Run bigger Repetition codes
52,2023-10-29 14:43:49.136614+01:00,bigger_rep_codes,ibm_sherbrooke,cmz635cvayrg008sp2ag,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,1,_is_hex=True,Run bigger Repetition codes


In [4]:
memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memories

{'mmr_log_0': array([[-1.07835540e+07+1.60007760e+07j, -4.34302570e+07+6.96242610e+07j,
         -7.66192810e+07-1.01656696e+08j, ...,
         -1.35120390e+08-9.77335290e+07j, -3.80560740e+07+6.36918740e+07j,
         -4.85890250e+07-4.91702220e+07j],
        [-1.74411220e+07+1.59327660e+07j, -2.78251510e+07+9.61105720e+07j,
         -8.27942290e+07-6.93373260e+07j, ...,
         -9.07659150e+07-8.55185450e+07j, -4.64208250e+07+6.32030470e+07j,
         -4.33537600e+07-6.12943710e+07j],
        [-1.54824580e+07+2.18220430e+07j, -6.89706470e+07+8.39149230e+07j,
         -8.66970110e+07-1.03769749e+08j, ...,
         -9.09095960e+07-1.00895483e+08j,  2.93957050e+07+5.42357540e+07j,
         -4.63021990e+07-4.44538090e+07j],
        ...,
        [-1.39799180e+07+2.69024710e+07j, -6.32237650e+07+8.82667040e+07j,
         -5.41600650e+07-7.07844530e+07j, ...,
         -1.03823050e+08-6.60265260e+07j, -3.28060750e+07+5.71764790e+07j,
         -4.21334650e+07-3.51620440e+07j],
        [-2.26

In [29]:
import pickle

from soft_info import get_repcode_layout, get_KDEs, get_counts
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit_qec.decoders import UnionFindDecoder

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_sherbrooke"), _is_hex=True)

kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sherbrooke', layout, bandwidths=0.2, plot=False)
print(kde_dict)

code = RepetitionCodeCircuit(max_distance, max_distance)
dcd_pkl_str = f"dist_{max_distance}_decoder.pkl"

{114: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 116: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 118: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 120: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 122: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 104: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 102: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 100: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 98: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 113: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 115: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 117: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 119: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 121: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 111: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.

from qiskit_qec.decoders.hdrg_decoders import UnionFindDecoder_v2

decoder = UnionFindDecoder_v2(code)
dcd_pkl_str = f"dist_{max_distance}_decoder_v2.pkl"

print("saving the decoder...")
with open(dcd_pkl_str, 'wb') as f:
    pickle.dump(decoder, f)

In [30]:
with open(dcd_pkl_str, 'rb') as f:
    decoder = pickle.load(f)

In [41]:
from soft_info import soft_reweight, get_counts, rx_draw_2D
from tqdm import tqdm

VERBOSE = False

memory = memories['mmr_log_1']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))[:]):
    
    with open(dcd_pkl_str, 'rb') as f: # Reload the decoder each time to reset the graph
        decoder = pickle.load(f)

    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    if VERBOSE:
        print("Decoding graph before reweighting:")
        rx_draw_2D(decoder.graph)

    reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout, common_measure=1)
    decoder.graph = reweighted_graph

    if VERBOSE:
        print("Decoding graph after reweighting:")
        rx_draw_2D(decoder.graph)

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    
    if VERBOSE:
        print("\nmeas_str:", count_key)
        print("flipped_qubit_dict", flipped_qubit_dict)
        print("logical:", logical, "\n")
        if logical[0] == 0:
            print("------------ Wrong logical! ------------\n")
            
    if logical[0] == 0:
            print("------------ Wrong logical! ------------\n")
    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)


  0%|          | 0/10000 [00:00<?, ?it/s]

 10%|▉         | 955/10000 [02:58<31:15,  4.82it/s]

------------ Wrong logical! ------------



 13%|█▎        | 1310/10000 [04:05<27:03,  5.35it/s]

------------ Wrong logical! ------------



 25%|██▍       | 2453/10000 [07:41<25:01,  5.03it/s]

------------ Wrong logical! ------------



 26%|██▌       | 2620/10000 [08:13<23:08,  5.31it/s]


KeyboardInterrupt: 

In [39]:
from soft_info import soft_reweight, get_counts, rx_draw_2D
from tqdm import tqdm

VERBOSE = False

memory = memories['mmr_log_1']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))[:]):
    
    with open(dcd_pkl_str, 'rb') as f: # Reload the decoder each time to reset the graph
        decoder = pickle.load(f)

    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    if VERBOSE:
        print("Decoding graph before reweighting:")
        rx_draw_2D(decoder.graph)

    #reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    #decoder.graph = reweighted_graph

    if VERBOSE:
        print("Decoding graph after reweighting:")
        rx_draw_2D(decoder.graph)

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    
    if VERBOSE:
        print("\nmeas_str:", count_key)
        print("flipped_qubit_dict", flipped_qubit_dict)
        print("logical:", logical, "\n")
        if logical[0] == 0:
            print("------------ Wrong logical! ------------\n")
    
    if logical[0] == 0:
            print("------------ Wrong logical! ------------\n")

    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)


  9%|▉         | 875/10000 [01:12<12:59, 11.71it/s]

------------ Wrong logical! ------------



 10%|▉         | 955/10000 [01:19<11:50, 12.73it/s]

------------ Wrong logical! ------------



 11%|█         | 1067/10000 [01:28<12:51, 11.57it/s]

...

------------ Wrong logical! ------------



 31%|███       | 3057/10000 [04:11<09:28, 12.21it/s]

------------ Wrong logical! ------------



 41%|████▏     | 4147/10000 [05:40<07:59, 12.20it/s]


KeyboardInterrupt: 